## Questões sobre Machine Learning

Dataset - pinguins

In [1]:
import pandas as pd

penguins = pd.read_csv("penguins.csv")

columns = ["Body Mass (g)", "Flipper Length (mm)", "Culmen Length (mm)"]
target_name = "Species"

# Remove lines with missing values for the columns of interestes
penguins_non_missing = penguins[columns + [target_name]].dropna()

data = penguins_non_missing[columns]
target = penguins_non_missing[target_name]

In [2]:
penguins.head()

,studyName,Sample Number,Species,Region,Island,Stage,Individual ID,Clutch Completion,Date Egg,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo),Comments
0,PAL0708,1,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A1,Yes,2007-11-11,39.1,18.7,181.0,3750.0,MALE,NaN,NaN,Not enough blood for isotopes.
1,PAL0708,2,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N1A2,Yes,2007-11-11,39.5,17.4,186.0,3800.0,FEMALE,8.94956,-24.69454,NaN
2,PAL0708,3,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A1,Yes,2007-11-16,40.3,18.0,195.0,3250.0,FEMALE,8.36821,-25.33302,NaN
3,PAL0708,4,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N2A2,Yes,2007-11-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adult not sampled.
4,PAL0708,5,Adelie Penguin (Pygoscelis adeliae),Anvers,Torgersen,"Adult, 1 Egg Stage",N3A1,Yes,2007-11-16,36.7,19.3,193.0,3450.0,FEMALE,8.76651,-25.32426,NaN


In [3]:
penguins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   studyName            344 non-null    object 
 1   Sample Number        344 non-null    int64  
 2   Species              344 non-null    object 
 3   Region               344 non-null    object 
 4   Island               344 non-null    object 
 5   Stage                344 non-null    object 
 6   Individual ID        344 non-null    object 
 7   Clutch Completion    344 non-null    object 
 8   Date Egg             344 non-null    object 
 9   Culmen Length (mm)   342 non-null    float64
 10  Culmen Depth (mm)    342 non-null    float64
 11  Flipper Length (mm)  342 non-null    float64
 12  Body Mass (g)        342 non-null    float64
 13  Sex                  334 non-null    object 
 14  Delta 15 N (o/oo)    330 non-null    float64
 15  Delta 13 C (o/oo)    331 non-null    flo

In [4]:
target.value_counts()

Adelie Penguin (Pygoscelis adeliae)          151
Gentoo penguin (Pygoscelis papua)            123
Chinstrap penguin (Pygoscelis antarctica)     68
Name: Species, dtype: int64

### Conclusões iniciais sobre o dataset:

1. O problema a ser solucionado é um problema classificação multiclasse.
2. A proporção de contagem das classes são desbalenceadas: algumas classes são 2x maior do que outra.

Selecionando apenas o dataset com variáveis numéricas:

In [5]:
data_numeric = data.select_dtypes(exclude=['object']).columns
data_numeric

Index(['Body Mass (g)', 'Flipper Length (mm)', 'Culmen Length (mm)'], dtype='object')

In [6]:
data = data[data_numeric].copy()
data.head()

,Body Mass (g),Flipper Length (mm),Culmen Length (mm)
0,3750.0,181.0,39.1
1,3800.0,186.0,39.5
2,3250.0,195.0,40.3
4,3450.0,193.0,36.7
5,3650.0,190.0,39.3


In [7]:
data.isna().sum()

Body Mass (g)          0
Flipper Length (mm)    0
Culmen Length (mm)     0
dtype: int64

In [9]:
data.shape

(342, 3)

Considere o seguinte pipeline:

1. Avalie um pipeline usando 10-fold cross-validation usando como métrica o scoring = 'balanced_accuracy.
2. Use o sklearn.model_selection.cross-validation com scoring='balanced_accuracy'. 
3. Use model.get_params() para listar os parâmetros do pipeline.
4. Use model.set_params(param_name = param_value) para atualizar-los.

Nós consideramos um modelo melhor (performance estatística) se sua média do score via cross-validation é maior do que a média + desvio padrão da cross-validation do que o modelo que estamos comparando.


In [15]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
model = Pipeline(steps=[
    ("preprocessor", StandardScaler()),
    ("classifier", KNeighborsClassifier(n_neighbors=5)),
])

In [16]:
print(f"Os steps do pipeline são: \n{model.steps}")

Os steps do pipeline são: 
[('preprocessor', StandardScaler()), ('classifier', KNeighborsClassifier())]


In [17]:
print("Os parâmetros do pipeline são:")
for parameter in model.get_params():
    print(f"{parameter}")

Os parâmetros do pipeline são:
memory
steps
verbose
preprocessor
classifier
preprocessor__copy
preprocessor__with_mean
preprocessor__with_std
classifier__algorithm
classifier__leaf_size
classifier__metric
classifier__metric_params
classifier__n_jobs
classifier__n_neighbors
classifier__p
classifier__weights


#### Implementando a validação cruzada com : cross-validation_curve
***

In [18]:
%%time
cv_results = cross_validate(
model, # pipeline
data, target, # feature matrix, target vector
scoring = "balanced_accuracy", # loss function
cv = 10, #cross-validation technique
n_jobs = -1,
)
scores = cv_results['test_score']
print(f"A média da acurácia no cross-validation é: {scores.mean():.3f} +/- {scores.std():.3f}")

A média da acurácia no cross-validation é: 0.952 +/- 0.040
CPU times: user 62.6 ms, sys: 64.2 ms, total: 127 ms
Wall time: 2.76 s


* A média da "balanced-accuracy" no conjunto de teste do pipeline acima é entre: $0.9 \lt 1.0 $

É possível mudar os parâmetros do pipeline e rodar novamente a validação cruzada da seguinte forma:

In [20]:
model.set_params(classifier__n_neighbors=51)
cv_results = cross_validate(
model, # pipeline
data, target, # feature matrix, target vector
scoring = "balanced_accuracy", # loss function
cv = 10, #cross-validation technique
n_jobs = -1,
)
scores = cv_results['test_score']
print(f"A média da acurácia no cross-validation é: {scores.mean():.3f} +/- {scores.std():.3f}")

A média da acurácia no cross-validation é: 0.942 +/- 0.039


Mudando o número de n_neighbors para 51, temos um score um pouco pior do que o obtido com 5. Podemos voltar para o valor anterior e setar o preprocessador para None com:

In [21]:
model.set_params(preprocessor=None, classifier__n_neighbors = 5)
cv_results = cross_validate(
model, # pipeline
data, target, # feature matrix, target vector
scoring = "balanced_accuracy", # loss function
cv = 10, #cross-validation technique
n_jobs = -1,
)
scores = cv_results['test_score']
print(f"A média da acurácia no cross-validation é: {scores.mean():.3f} +/- {scores.std():.3f}")

A média da acurácia no cross-validation é: 0.740 +/- 0.087


Resumindo, não fazendo preprocessando com o algoritmo k_neighbors, temos um score bem pior. Comprovando que este algoritmo necessita que as variáveis numéricas estejam preprocessadas.

Para a próxima questão iremos estudar o impacto de diferentes tipos de processadores no pipeline do modelo definido inicialmente.

* MinMaxScaler
* QuantileTranformer
* PowerTransformer

O método Box-Cox é uma estratégia comum para valores positivos. Os outros preprocessadores funcionam para qualquer tipo de feature numérica.

Use o sklearn.model_selection.GridSearchCV para estudar o impacto da escolha do preprocessador e o número de neighbors em uma validação cruzada com cv = 10, usando a métrica scoring = 'balanced_accuracy'. Queremos estudar a influência do range n_neighbors = [5, 51, 101] com todos os preprocessadores.

In [22]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer


all_preprocessors = [
    None,
    StandardScaler(),
    MinMaxScaler(),
    QuantileTransformer(n_quantiles=100),
    PowerTransformer(method="box-cox"),
]

In [23]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "preprocessor": all_preprocessors,
    "classifier__n_neighbors": [5, 51, 101],
}

grid_search = GridSearchCV(
    model,
    param_grid=param_grid,
    scoring="balanced_accuracy",
    cv=10,
).fit(data, target)
grid_search.cv_results_

{'mean_fit_time': array([0.00517299, 0.00633981, 0.00551231, 0.00772493, 0.01412013,
        0.00616221, 0.00761209, 0.00607443, 0.00771194, 0.01290171,
        0.00451956, 0.00806606, 0.00820496, 0.00961752, 0.01369481]),
 'std_fit_time': array([0.00162966, 0.00090139, 0.00019069, 0.00096377, 0.0026218 ,
        0.00092373, 0.00101022, 0.00095238, 0.00095634, 0.00187563,
        0.0006682 , 0.00076931, 0.00107406, 0.00082228, 0.00191603]),
 'mean_score_time': array([0.00736599, 0.00620892, 0.00620825, 0.00643232, 0.00766938,
        0.00818155, 0.00721512, 0.0065613 , 0.00633509, 0.00702453,
        0.00727072, 0.00863185, 0.00870481, 0.00802302, 0.00763559]),
 'std_score_time': array([0.00094419, 0.00104062, 0.00076182, 0.00109847, 0.0011511 ,
        0.00106737, 0.00129363, 0.00080641, 0.00043057, 0.00104278,
        0.00084532, 0.00104403, 0.00178715, 0.00053528, 0.00091649]),
 'param_classifier__n_neighbors': masked_array(data=[5, 5, 5, 5, 5, 51, 51, 51, 51, 51, 101, 101, 101, 101

Podemos sortear os resultados e focar apenas nas colunas de interesse:

In [24]:
results = (
    pd.DataFrame(grid_search.cv_results_)
    .sort_values(by="mean_test_score", ascending=False)
)

results = results[
    [c for c in results.columns if c.startswith("param_")]
    + ["mean_test_score", "std_test_score"]
]


In [25]:
results.shape

(15, 4)

In [26]:
results

,param_classifier__n_neighbors,param_preprocessor,mean_test_score,std_test_score
1,5,StandardScaler(),0.952198,0.039902
2,5,MinMaxScaler(),0.947778,0.034268
3,5,QuantileTransformer(n_quantiles=100),0.947094,0.033797
4,5,PowerTransformer(method='box-cox'),0.946960,0.047387
6,51,StandardScaler(),0.941880,0.038905
8,51,QuantileTransformer(n_quantiles=100),0.927277,0.043759
9,51,PowerTransformer(method='box-cox'),0.922833,0.047883
7,51,MinMaxScaler(),0.920293,0.045516
11,101,StandardScaler(),0.876642,0.041618
12,101,MinMaxScaler(),0.862357,0.046244


Podemos observar que os melhores scores são para n_neighbors = 5, com qualquer um dos processadores. Em seguida temos n_neighbors = 51 com uma performance estatística um pouco pior, mas não exatamente muito distante.Para tais modelos temos uma média de acurácia no conjunto de testes acima de $0.92$ enquanto o melhor modelo é em torno de $0.95 \pm 0.04$.

Os modelos sem processamento (preprocessor = None) são todos abaixo de 0.75, mesmo com **n_neighbors=5**.

Modelos com qualquer processamento e n_neighbors=101 estão entre [0.80, 0.88]. Estes são significante melhores do que os modelos sem processamento algum, mas piores do que os modelos com um valor de n_neighbors = 5.

A principal razão que explica remover o preprocessamento leva a performance ruins é que as features de entrada tem range de valores bem diferentes qdo usam as unidades padrões (g, mm).

Usualmente, setando valores grandes de n_neighbors, faz os modelos underfitting. Aqui o dado é bem estruturado tem muito pouco ruído: usando valores de n_neighbord é bom, ou melhor do que valores intermediários que não overfitting.